In [1]:
%load_ext autoreload
%autoreload 2

from google.colab import drive
drive.mount('/content/drive/')
%cd '/content/drive/My Drive/aqc/dev'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/physics/aqc/dev


## Vacuum propagation

In [2]:
import numpy as np
from aqc.aqc import AQC
from aqc.grid import RectGrid, RandLogPolarGrid
from aqc.sources import GaussianSource
from aqc.pathes import VacuumPath, IdenticalPhaseScreensPath
from aqc.phase_screens import FFTPhaseScreen, SUPhaseScreen
from aqc.theory.models import MVKModel, KModel, TModel

from aqc.measures import mean_r2, I, eta

In [3]:
sim_vacuum = AQC(
  grid=RectGrid(
    resolution=2**8, 
    delta=0.002
  ), 
  source=GaussianSource(
    wvl=809e-9, 
    w0=0.02, 
    F0=np.inf
  ),
  path=VacuumPath(
    length=4e3
  ),
  use_GPU=True
)

In [4]:
def test_full_eta(sim):
  actual_eta = eta(sim, pupil=False)
  np.testing.assert_almost_equal(actual_eta, 1, decimal=6)

test_full_eta(sim_vacuum)

In [5]:
def test_norm_amplitude(sim):
  actual_norm = abs(sim.run(pupil=False).sum()) * sim.grid.delta**2
  desired_norm = np.sqrt(2 * np.pi) * sim.source.w0
  np.testing.assert_almost_equal(actual_norm, desired_norm, decimal=7)

test_norm_amplitude(sim_vacuum)

In [6]:
def test_norm_gamma2(sim):
  actual_norm = I(sim, pupil=False).sum() * sim.grid.delta**2
  np.testing.assert_almost_equal(actual_norm, 1, decimal=5)

test_norm_gamma2(sim_vacuum)

In [7]:
def test_vacuum_propagated_colim_waist(sim):
  def waist(z, w0, wvl):
    zR = np.pi * w0**2 / wvl
    return w0 * np.sqrt(1 + (z/zR)**2)

  length = sim.path.length
  w0 = sim.source.w0
  wvl = sim.source.wvl

  desired_waist = waist(length, w0, wvl)
  actual_waist = np.sqrt(2 * mean_r2(sim))
  np.testing.assert_almost_equal(actual_waist, desired_waist, decimal=7)

test_vacuum_propagated_colim_waist(sim_vacuum)